In [1]:
import fitz
from pix2text import Pix2Text

[WARNING 2025-07-23 23:47:06,080 _showwarnmsg:109] d:\User\ProjectGithub\hiepnguyenn-99\RAG-Solve-Math\ragmathenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
 


In [3]:
import onnxruntime as ort
print(ort.get_device())

GPU


# giao_trinh_4.pdf

In [ ]:
# trích suất trang đầu phần 1.1 Các khái niệm cơ bản

# kiểm tra giao nhau
def intersects(b1, b2):
    x0, y0, x1, y1 = b1
    X0, Y0, X1, Y1 = b2
    return not (x0 > X1 or y0 > Y1 or x1 < X0 or y1 < Y0)

# phát hiện công thức và chuyển thành latex
def extract_text_and_formula(page):
    layout = page.get_text(
        'dict',
        sort=True
        )
    pix = page.get_pixmap(dpi=300)
    img = pix.pil_image() # chuyển về ảnh pil
    
    dpi = 300
    zoom = dpi/72.0 # 1 point = 1/72 inch (pt->px)
    mat_forward = fitz.Matrix(zoom, zoom)
    mat = ~mat_forward # ma trận nghịch đảo (px->pt)

    # phát hiện công thức trong ảnh
    page_obj = p2t.recognize_page(
        img,
        file_type='text_formula',
        resized_shape=1024,
        return_text=False,
        save_debug_res=None
    )
    formulas = [e for e in page_obj.elements if e.type.name == 'FORMULA']

    seen_formula = set()
    for f in formulas:
        r_px = fitz.Rect(f.box) # region (x0, y0, x1, y1)
        r_pt = r_px * mat
        latex = f.text
        # lấy khối văn bản
        for block in layout['blocks']:
            # lấy dòng văn bản trong khối
            for line in block.get('lines', []):
                kept = []
                for span in line['spans']:
                    sb = fitz.Rect(span['bbox'])
                    overlap = abs(sb & r_pt)
                    # giữ lại spans có phần giao với công thức ko quá 80% span
                    if overlap < 0.8 * abs(sb):
                        kept.append(span)
                    else:
                        if not latex in seen_formula:
                            kept.append({
                                'type': 'formula',
                                'text': f'$$ {latex} $$',
                                'bbox': r_pt
                            })
                            seen_formula.add(latex)
                line['spans'] = kept
    return layout

# chuyển pdf về markdown
def render_md(layout):
    md = ''
    for block in layout['blocks']:
        for line in block.get('lines', []):
            spans = sorted(line['spans'], key=lambda s: s['bbox'][0])
            for s in spans:
                md += s['text']
            md += '\n'
        md += '\n'
    return md

p2t = Pix2Text.from_config(
    analyzer_config={
        "model_name": "mfd-pro",
        "model_backend": "onnx"
    },
    formula_ocr_config={
        "model_name": "mfr-pro",
        "model_backend": "onnx"
    }
)

doc = fitz.open('source/giao_trinh_4.pdf')
page = doc.load_page(11)
layout = extract_text_and_formula(page)
md = render_md(layout)
print(md)

[INFO] 2025-07-23 23:47:11,815 [RapidOCR] base.py:24: Using engine_name: onnxruntime
[INFO] 2025-07-23 23:47:11,815 [RapidOCR] main.py:55: Using C:\Users\admin\AppData\Roaming\cnstd\1.2\ppocr\ch_PP-OCRv5_det\ch_PP-OCRv5_det_infer.onnx
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.



0: 1024x736 9 titles, 11 plain texts, 1 abandon, 5 isolate_formulas, 1341.7ms
Speed: 14.8ms preprocess, 1341.7ms inference, 0.0ms postprocess per image at shape (1, 3, 1024, 736)
Loading C:\Users\admin\AppData\Roaming\pix2text\1.1\mfd-onnx\mfd-v20240618.onnx for ONNX Runtime inference...
Using ONNX Runtime CPUExecutionProvider

WARNING imgsz=[617, 1024] must be multiple of max stride 32, updating to [640, 1024]
0: 640x1024 1 embedding, 284.8ms
Speed: 5.0ms preprocess, 284.8ms inference, 1.8ms postprocess per image at shape (1, 3, 640, 1024)


100%|██████████| 1/1 [00:00<00:00,  4.95it/s]



WARNING imgsz=[111, 1536] must be multiple of max stride 32, updating to [128, 1536]
0: 128x1536 1 embedding, 59.9ms
Speed: 1.2ms preprocess, 59.9ms inference, 15.2ms postprocess per image at shape (1, 3, 128, 1536)


100%|██████████| 1/1 [00:03<00:00,  3.30s/it]

...


WARNING imgsz=[186, 1024] must be multiple of max stride 32, updating to [192, 1024]
0: 192x1024 (no detections), 63.6ms
Speed: 1.4ms preprocess, 63.6ms inference, 0.7ms postprocess per image at shape (1, 3, 192, 1024)



0it [00:00, ?it/s]



WARNING imgsz=[1447, 1024] must be multiple of max stride 32, updating to [1472, 1024]
0: 1472x1024 (no detections), 531.7ms
Speed: 13.6ms preprocess, 531.7ms inference, 1.2ms postprocess per image at shape (1, 3, 1472, 1024)


0it [00:00, ?it/s]


Chương 1

Ma trận

Nội dung

• Định nghĩa và ví dụ.

• Các phép biến đổi sơ cấp.

• Các phép toán đối với ma trận.

• Hạng của ma trận.

• Ma trận nghịch đảo.

1.1
Các khái niệm cơ bản

Định nghĩa 1.1 (Ma trận) .
Ma trận cỡm × n là một bảng số(thực hoặc phức) hình chữnhật có m hàng và n cột.

$$ A=\left( \begin{matrix} {a_{1 1}} & {\cdots} & {a_{1 j}} & {\cdots} & {a_{1 n}} \\ {\cdots} & {\cdots} & {\cdots} & {\cdots} & {\cdots} \\ {a_{i 1}} & {\cdots} & {a_{i j}} & {\cdots} & {a_{i n}} \\ {\cdots} & {\cdots} & {\cdots} & {\cdots} & {\cdots} \\ {a_{m 1}} & {\cdots} & {a_{m j}} & {\cdots} & {a_{m n}} \\ \end{matrix} \right)
 $$











































Ví dụ1.1

$$ A=\left( \begin{matrix} {3} & {4} & {1} \\ {2} & {0} & {5} \\ \end{matrix} \right)_{2 \times3}, B=\left( \begin{matrix} {1+i} & {2} \\ {3-i} & {4 i} \\ \end{matrix} \right)
 $$


















A là ma trận cỡ2 × 3 có 2 hàng và 3 cột. Các phần tửcủa ma trận A:

$$ a_{1 1}=3, a_{1 2}=4, a_{1 3}=1, a_{2 1}=2,

In [ ]:
with open('extracted/giao_trinh_4.md', 'w', encoding='utf-8') as f:
    for line in md.splitlines():
        if line.strip():
            f.write(line + '\n')